In [ ]:
import numpy as np
import numpy #数组相关
import keras #神经网络相关
from scipy import signal #预处理
import os #环境
import tensorflow as tf #神经网络框架
from matplotlib import pyplot as plt #绘图
from obspy import read #数据读取
import sys, getopt #环境
import time #计时
import matplotlib #绘图
from matplotlib.figure import Figure #绘图
import h5py #读取文件 


In [13]:
model = keras.models.load_model('mini2.h5')
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_29 (Conv2D)          (None, 512, 512, 8)       400       
                                                                 
 leaky_re_lu_48 (LeakyReLU)  (None, 512, 512, 8)       0         
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 128, 128, 8)      0         
 g2D)                                                            
                                                                 
 conv2d_30 (Conv2D)          (None, 128, 128, 8)       1608      
                                                                 
 leaky_re_lu_49 (LeakyReLU)  (None, 128, 128, 8)       0         
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 32, 32, 8)        0         
 g2D)                                                 

In [ ]:
#network 输入维度大小
trace_win=512 
time_win=512
#network 训练时的采样率（不要修改，除非你知道这意味着什么）
time_fre=100
#带通滤波的值
low_freq=20
hign_freq=25
#检测窗口的重叠率
overlap=0.5
#连续记录的路径（请保持所有记录具有相同的道数、采样率、文件长度等，以下所有代码不包括动态判断文件格式是否变化；如果需要可以修改读取函数并且在预处理函数中进行相应的修改）
path=''
#读取路径中符合条件的文件
files = [name for name in os.listdir( path) if name.endswith('.mat')]
#排序（注意，如果你的文件名按照一般排序后，时间不是连续的，那么文件的拼接会出现大问题，并且不会报错）
files.sort()
#文件长度
file_lenth=len(files)
#文件位置指针
file_point=-1
#时间位置指针
time_point=0
#结尾判断
is_end=0
#记录的道数（低于 512 时程序不会正常运行，如果需要，可以找我更改）
channels=0
#记录的采样率
freq=0
#记录的长度
time_lenth=0
#重采样比率
time_scale=0
#备用的参数
count_fp=0
#这个函数可以自动读取列表中的第一个文件，并且计算需要的参数（mat 格式）
def get_parameters(path,files,time_fre):
    mat = h5py.File(path+files[0])
    matdata=mat['data']
    freq=1/mat['dt'][0,0]
    channels=matdata.shape[0]
    times_lenth=mat['npts'][0,0]
    time_scale=time_fre/freq
    return freq,channels,times_lenth,time_scale
#这个函数可以自动读取列表中的第一个文件，并且计算需要的参数（sgy 格式）
def get_parameters(path, files, time_fre):
    data=read(path+files[0])
    freq=data[0].stats['sampling_rate']
    channels=len(data)
    times_lenth=data[0].stats['npts']
    time_scale=time_fre / freq
    return freq, channels, times_lenth, time_scale
#这个函数可以自动读取列表中的第一个文件，并且计算需要的参数（dat 格式）
def get_parameters(path,files,time_fre):
    file = numpy.fromfile(path+files[0], dtype="f4")
    header = file[:10]
    freq = int(header[6])
    times_lenth = int(header[7])
    channels = int(header[9])
    time_scale=time_fre/freq
    return freq,channels-1,times_lenth,time_scale
 
freq,channels,time_lenth,time_scale=get_parameters(path,files,time_fre)


In [ ]:
def sgy2numpy(file):
    global channels
    temp=[]
    temp_data=read(file)
    for i in range(channels):
        temp.append(temp_data[i].data)
    return numpy.array(temp)
def mat2numpy(file):
    global channels,time_lenth
    mat = h5py.File(file)
    data=mat['data']
    d=numpy.zeros_like(data)
    for i in range(data.shape[0]-1):
        d[i]=data[i+1]-data[i]
    return d
def dat2numpy(filename):
    file = numpy.fromfile(filename, dtype="f4")
    header = file[:10]
    fs = int(header[6])
    npts = int(header[7])
    dt = 1/fs
    nch = int(header[9])
    data = file[10:].reshape((nch,npts))
    return data


In [ ]:
def data_filter(data,fre_min,fre_max):
    fmin=fre_min/(freq/2)
    fmax=fre_max/(freq/2)
    #print(fmin,fmax)
    s1,s2=signal.butter(3,[fmin,fmax],'bandpass')
    data_=signal.filtfilt(s1,s2,data) .copy()
    return data_
def data_S_L(data,freq):
    #此处需要全局变量
    short=int(0.2*freq)
    long=int(freq)
    #print('S_L:',short,long)
    data_=data.copy()
    for i in range(data.shape[0]):
        data_[i,:]=S_L(data_[i,:],short,long).copy()
    return data_[:,long: -short]


In [ ]:
def file_input():
    global file_point,low_freq,hign_freq,freq,files,path
    #文件指针后移
    file_point=file_point+1
    print('file_input:'+str(file_point)+' '+files[file_point][21:31])
    file_=path+files[file_point]
    #调用文件读取函数，这里需要自行更改想要使用的函数
    #这里没有异常检测函数，如果需要可以自己增加
    data_=mat2numpy(file_)
    return data_


In [ ]:
def init(begin):
    #begin 指定开始的地方，例如，在程序运行到第 888 个文件时突然中断，可以将 begin 设为 888 接着开始
    global file_point,low_freq,hign_freq,freq
    global buffer_num
    file_point=begin
    #接收读取的数组
    data_init=file_input().copy()
    l1,l2=data_init.shape
    buffer=numpy.zeros((l1,buffer_num*l2))
    buffer[:,0:l2]=data_init.copy()
    #塞满缓冲区
    for i in range(buffer_num-1):
        data_init=file_input().copy()
        buffer[:,(i+1)*l2:(i+2)*l2]=data_init.copy()
    return buffer
def buffer_update(buffer):
    global file_point,time_point,is_end,file_lenth
    global buffer_num
    #这里判断是否已经扫完所有的文件
    if file_point+buffer_num > file_lenth:
        is_end=1
    else:
        #将最后的一个文件提到最前方，这里是为了避免最后的部分出现目标信号，但是由于截断而漏检测
        temp_each=int(buffer.shape[1]/buffer_num)
        buffer[:,0:temp_each]=buffer[:,-temp_each:].copy()
        #塞满后续的缓冲区
        for i in range(buffer_num-1):
            data_=file_input().copy()
            buffer[:,(i+1)*temp_each:(i+2)*temp_each]=data_.copy()
    return buffer
def data_get(buffer):
    global file_point,time_point,is_end,file_lenth,time_win,time_scale,trace_win,freq
    global overlap
    _overlap=1-overlap
    #滤波
    buffer=data_filter(buffer,low_freq,hign_freq)
    #长短窗
    buffer=data_S_L(buffer,freq)/5
    #重采样
    #这里的重采样方案可以保证即使输入通道少于 512 也能良好的运行，但是速度较慢
    buffer=signal.resample(buffer,int(buffer.shape[1]*time_scale),axis=1)
    buffer=signal.resample(buffer,512,axis=0).copy()
#-------------------------- 更高效的方法是抽样：----------------------------#
    '''
    buffer_=data_filter(buffer,low_freq,hign_freq)
    buffer_=data_f[numpy.arange(512)*channels//512]
    buffer_=buffer_[:,(numpy.arange(buffer_.shape[1]*time_scale)*int(1/time_scale)).astype('int')]
    buffer_=data_S_L(buffer_,freq/5).copy()/5
    '''
#------------------------ 或者可以采用叠加替换抽样提高检测能力 -----------------------------#
    #print(buffer.shape)
    win_num=int((buffer.shape[1]-overlap*time_win)//(_overlap*time_win)+1)
    #print(win_num)
    data_for_nn=numpy.zeros((win_num,time_win,trace_win))
    #这里是切割一个个的检测窗口
    for i in range(win_num-1):
        data_for_nn[i,:,:]=buffer[:,int(i*time_win*_overlap):int(i*time_win*_overlap+time_win)]
    data_for_nn[win_num-1,:,:]=buffer[:,(buffer.shape[1]-time_win):]
    return data_for_nn


In [ ]:
def detec_flow(path,times=1e10,begin=0):
    global model,model2,file_point,files
    file_point=begin
    #初始化
    buffer=init(begin)
    time_=numpy.zeros(min(times,len(files)))
    t0=time.time()
    print('')
    #循环检测
    while is_end==0 and file_point<times:
        #可以添加用时记录
        '''
        if file_point%10==0:
            print(data_each.shape)
            print('time per file between',file_point-10,'-',file_point,':',(time.clock()-t0)/10)
        '''
        data_each=data_get(buffer)
        #这里使用了两个模型，所以写了两遍
        ans=data_detec(data_each,model)
        ans2=data_detec(data_each,model2)
        #ans2=model2.predict(data_each[:,:,:,0:1])
        print(file_point)
        is_true(ans,ans2,buffer,data_each)
        time_[file_point]=time.time()-t0
        #print('speed:',(time.time()-t0)/(file_point-begin),'cost:',(time.time()-t0))
        buffer=buffer_update(buffer)
    return time_


In [ ]:
def is_true(ans,ans_,buffer,data_each):
    global freq,count_fp,buffer2
    global eq_files,full_files
    print(buffer2.shape)
    plt.imshow(buffer2,aspect='auto')
    plt.xticks(numpy.arange(0,time_lenth*buffer_num*time_scale,time_lenth*time_scale),numpy.arange(0,int(time_lenth/freq*buffer_num),int(time_lenth/freq)))
    plt.yticks([0,256,512],[0,256*4,512*4])
    plt.xlabel('Time (s)')
    plt.ylabel('Channel')
    count_fp=count_fp+1
    for i in range(len(ans)-1):
        begin=i*time_win*(1-overlap)
        #print(begin)
        plt.plot([begin,begin+512],[512-i*7,512-i*7],color=colors[numpy.argmax(ans[i])])
        plt.plot([begin,begin+512],[452-i*7,452-i*7],color=colors[numpy.argmax(ans_[i])])
        #plt.plot([begin,begin+512/time_scale],[7000-i*100,7000-i*100],color=colors2[numpy.argmax(ans2[i])])
    #plt.ylim(0,channels)
    #plt.xlim(0,time_lenth*buffer_num-250)
    plt.grid(axis='x',ls='--',color='white')
    name=files[file_point-buffer_num+1][5:-8]
    plt.title(name)
    plt.savefig('./'+full_files+'/'+str(file_point-buffer_num+1)+'__'+name,dpi=100)
    #print(ans)
    if ans[:,1].max()>0.5 or ans_[:,1].max()>0.5:
        plt.savefig('./'+eq_files+'/'+str(file_point-buffer_num+1)+'__'+name,dpi=100)
        numpy.save('/home/lh21/DATB/APMusers/lvhao/qinghai_output/'+name+'__'+str(file_point-buffer_num+1),buffer2.astype('float16'))
    plt.show()
    return 1


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
#用到模型权重文件
model2=keras.models.load_model('./mini2_qinghai.h5')
model=keras.models.load_model('./mini2_for_qinghai.h5')
#画图用到的颜色
colors=['white','red']
path='/home/lh21/DATA/Menyuan/Qingshizui/old/data/2022-01-09/'
files = [name for name in os.listdir( path) if name.endswith('.dat')]
files.sort()
file_lenth=len(files)
freq,channels,time_lenth,time_scale=get_parameters(path,files,time_fre)
 
# 这是 istrue 函数里用到的，分别保存有地震信号和全部检测记录
eq_files='./09eq_output'
full_files='./09full_output'
createFile(eq_files)
createFile(full_files)
buffer_num=5
times=1e5
overlap=0.5
file_point=0
print(overlap)
is_end=0
 
#这里开始检测
detec_flow(path,times,begin=452)
